In [132]:
# Import Packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, r2_score


In [133]:
# Inputs

cell_doubling = 17.0                   # cell doubling time (h)
siRNA0 = 1500                          # transfection concentration (nM)
transfection_time = 0.08               # time that the cells are exposed to siRNA (h)
V_ex = 100 * 10 ** -6                  # transfection volume (L)
V_in = 2000 * 10 ** -15                # Intracellular volume (L)

# Conversions

mol_M = 6.022 * 10 ** 23               # (number/mole)
rtot = 1.9 * 10 ** 15                  # (number/L)
r_tot = rtot/mol_M * (10 ** 9)         # nM

# Rates

siRNA_half_life = 24.0                 # siRNA half-life (h)
mRNA_half_life = 7                     # mRNA half-life (h)
prot_half_life = 26                    # protein half-life (h)

k_sdeg = 0.69315 / siRNA_half_life     # (h^-1)
k_sdil = 0.69315 / cell_doubling       # (h^-1)
k_mRNA = 0.69315 / mRNA_half_life      # (h^-1)
k_mdeg = 0.69315 / mRNA_half_life      # (h^-1)
k_prot = 0.69315 / prot_half_life      # (h^-1)
k_pdeg = 0.69315 / prot_half_life      # (h^-1)

k_risc_orig = 2 * 10 ** -19            # (L/h/number)
k_risc_M = k_risc_orig * mol_M         # (M^-1 h^-1)
k_risc = k_risc_M * 10 ** -9           # (nM^-1 h^-1)

k_cleav = 5 * 10 ** 2                  # (nm^-1 h^-1)

k_rdeg = 0.077                         # (h^-1)

In [134]:
# Time and steps

t0 = 0
tf_transfection = 200.0                # total time of experiment?? (h)
dt = 0.01                              # time step (h)
t = np.linspace(t0,tf_transfection,int(tf_transfection/dt + dt))
n = len(t)


In [135]:
# Experimental Measurements
x = [48]                               # time of measurement (h)
y = [0.49]                             # relative gene expression levels

# changing time points to array for optimization of code
x_array = np.array(x)
x_opt_array = x_array * 100
x_indices_to_access = x_opt_array.tolist()
print(x_indices_to_access)

[4800]


In [136]:
# Initial guess for internalization rate constant
k_int_init = 2.5 * 10 ** -11 # h^-1

# Define empty array for internalization rate
optimized_kint = 0

# Initial concentrations
z0 = [siRNA0,0,0,100,100]               # External siRNA, intracellular siRNA, RISC, mRNA, prot
DNA = 100

# Unpopulated lists
siRNA_ex = np.ones(n) * z0[0]
siRNA = np.ones(n) * z0[1]
RISC = np.ones(n) * z0[2]
mRNA = np.ones(n) * z0[3]
prot = np.ones(n) * z0[4]

In [137]:
# optimization code

def kint_rmse(k_int):
    for i in range(1,n):
        siRNA_ex[i] = (-k_int * siRNA_ex[i-1]) * dt + siRNA_ex[i-1]
        if i > (transfection_time / dt):
            siRNA_ex[i] = 0
        siRNA[i] = ((k_int * (V_ex/V_in) * siRNA_ex[i-1]) - k_sdeg * siRNA[i-1] - k_sdil * siRNA[i-1]) * dt + siRNA[i-1]
        RISC[i] = (k_risc * siRNA[i-1] * (r_tot - RISC[i-1]) - k_rdeg * RISC[i-1]) * dt + RISC[i-1]
        if RISC[i] > r_tot:
            RISC[i] = r_tot
        mRNA[i] = (k_mRNA * DNA - k_mdeg * mRNA[i-1] - k_cleav * mRNA[i-1] * RISC[i-1]) * dt + mRNA[i-1]
        if mRNA[i] < 0:
            mRNA[i] = 0
        prot[i] = (k_prot * mRNA[i-1] - k_pdeg * prot[i-1]) * dt + prot[i-1]
        if prot[i] < 0:
            prot[i] = 0
        prot_norm = prot / z0[4]
        prot_norm_array = np.array(prot_norm)
        
    y_pred_array = prot_norm_array[x_indices_to_access]
    y_pred = list(y_pred_array)
    rmse = np.sqrt(mean_squared_error(y,y_pred))
        
    return rmse
    
i = kint_rmse(k_int_init)
updated_k_int = k_int_init + (0.1 * 10 ** -11)
j = kint_rmse(updated_k_int)

if i < j:
    optimized_k_int = updated_k_int
else:
    while i > j:
        i = j
        optimized_k_int = updated_k_int
        updated_k_int = updated_k_int + (0.1 * 10 ** -11)
        j = kint_rmse(updated_k_int)
    
print(optimized_k_int)
print(i)

7.200000000000004e-11
0.0008019945634711867
